# 端到端建模流程
## 从原始数据输入到结果文件生成

# 0.引入三方库

In [1]:
import gc
import numpy as np
import pandas as pd
import lightgbm as lgb

# 1.读取所需数据

In [2]:
debug = False
NROWS = 10000 if debug else None
train = pd.read_csv('data/train.csv')
test =  pd.read_csv('data/test.csv')

merchant = pd.read_csv('data/merchants.csv')
new_transaction = pd.read_csv('data/new_merchant_transactions.csv', nrows = NROWS)
history_transaction = pd.read_csv('data/historical_transactions.csv', nrows = NROWS)

# 2.做数据预处理

In [3]:
# 商户信息
def change_object_cols(se):
    value = se.unique().tolist()
    value.sort()
    return se.map(pd.Series(range(len(value)), index=value)).values

In [4]:
category_cols = ['merchant_id', 'merchant_group_id', 'merchant_category_id',
       'subsector_id', 'category_1',
       'most_recent_sales_range', 'most_recent_purchases_range',
       'category_4', 'city_id', 'state_id', 'category_2']
numeric_cols = ['numerical_1', 'numerical_2',
     'avg_sales_lag3', 'avg_purchases_lag3', 'active_months_lag3',
       'avg_sales_lag6', 'avg_purchases_lag6', 'active_months_lag6',
       'avg_sales_lag12', 'avg_purchases_lag12', 'active_months_lag12']

# 将离散字段进行编码
for col in ['category_1', 'most_recent_sales_range', 'most_recent_purchases_range', 'category_4']:
    merchant[col] = change_object_cols(merchant[col])
    
# 同时为了能够更方便统计，首先做缺失值处理，对离散字段可统一用-1进行填充
merchant[category_cols] = merchant[category_cols].fillna(-1)

# 对离散型字段探查发现有正无穷值，这也是模型所不能接受的，因此需要对无限值进行处理
inf_cols = ['avg_purchases_lag3', 'avg_purchases_lag6', 'avg_purchases_lag12']
merchant[inf_cols] = merchant[inf_cols].replace(np.inf, merchant[inf_cols].replace(np.inf, -99).max().max())

# 对于离散字段的缺失值处理方式也有多样，这里先使用平均值进行填充，后续有需要再进行优化处理
for col in numeric_cols:
    merchant[col] = merchant[col].fillna(merchant[col].mean())
    
# 去除与transaction重复的列(当然, id显然不包括在内)
duplicate_cols = ['merchant_id', 'merchant_category_id', 'subsector_id', 'category_1', 'city_id', 'state_id', 'category_2']

In [5]:
merchant = merchant.drop(duplicate_cols[1:], axis=1)

merchant = merchant.loc[merchant['merchant_id'].drop_duplicates().index.tolist()].reset_index(drop=True)

In [6]:
# 交易记录
transaction = pd.concat([new_transaction, history_transaction], axis=0, ignore_index=True)
del new_transaction
del history_transaction
gc.collect()

numeric_cols = [ 'installments', 'month_lag', 'purchase_amount']
category_cols = ['authorized_flag', 'card_id', 'city_id', 'category_1',
       'category_3', 'merchant_category_id', 'merchant_id', 'category_2', 'state_id',
       'subsector_id']
time_cols = ['purchase_date']

In [7]:
# 可仿照merchant的处理方式对字符型的离散特征进行编码（id除外）
for col in ['authorized_flag', 'category_1', 'category_3']:
    transaction[col] = change_object_cols(transaction[col].fillna(-1).astype(str))
transaction[category_cols] = transaction[category_cols].fillna(-1)
transaction['category_2'] = transaction['category_2'].astype(int)
# 进行时间段的处理，简单起见进行月份、日期的星期数（工作日与周末）、以及
# 时间段（上午、下午、晚上、凌晨）进行提取

In [8]:
from datetime import datetime
transaction['purchase_month'] = transaction['purchase_date'].apply(lambda x:'-'.join(x.split(' ')[0].split('-')[:2]))
transaction['purchase_hour_section'] = transaction['purchase_date'].apply(lambda x: x.split(' ')[1].split(':')[0]).astype(int)//6
transaction['purchase_day'] = transaction['purchase_date'].apply(lambda x: datetime.strptime(x.split(" ")[0], "%Y-%m-%d").weekday())//5                                                                    

In [9]:
del transaction['purchase_date']

In [10]:
# 减少内存起见，统一对merchant和transaction的id列进行编码
merchant_id = set(merchant['merchant_id'].unique().tolist() + transaction['merchant_id'].unique().tolist())
merchant_map = pd.Series(range(len(merchant_id)), index=merchant_id)

In [11]:
transaction['merchant_id'] = transaction['merchant_id'].map(merchant_map)
merchant['merchant_id'] = merchant['merchant_id'].map(merchant_map)

In [12]:
# 减少内存起见，统一对merchant和transaction的id列进行编码

se_map = change_object_cols(train['first_active_month'].append(test['first_active_month']).astype(str))

train['first_active_month'] = se_map[:train.shape[0]]
test['first_active_month'] = se_map[train.shape[0]:]



In [13]:
## 对购买月份进行编码
transaction['purchase_month'] = change_object_cols(transaction['purchase_month'].fillna(-1).astype(str))

In [14]:
transaction.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
       'purchase_amount', 'category_2', 'state_id', 'subsector_id',
       'purchase_month', 'purchase_hour_section', 'purchase_day'],
      dtype='object')

In [15]:
## 初步策略，第一步采用字典结构统计用户的所有行为特征
## 第二步，用户层面横向进行比较和特征提取
## 第三步，纵向对比所有用户的交易记录
cols = ['merchant_id', 'most_recent_sales_range', 'most_recent_purchases_range', 'category_4']
transaction = pd.merge(transaction, merchant[cols], how='left', on='merchant_id')
numeric_cols = ['purchase_amount', 'installments']
category_cols = ['authorized_flag', 'city_id', 'category_1',
       'category_3', 'merchant_category_id','month_lag','most_recent_sales_range',
                 'most_recent_purchases_range', 'category_4',
                 'purchase_month', 'purchase_hour_section', 'purchase_day']
id_cols = ['card_id', 'merchant_id']

In [16]:
transaction[cols[1:]] = transaction[cols[1:]].fillna(-1).astype(int)

In [17]:
transaction[category_cols].nunique()

authorized_flag                  2
city_id                        308
category_1                       2
category_3                       4
merchant_category_id           331
month_lag                       16
most_recent_sales_range          6
most_recent_purchases_range      6
category_4                       3
purchase_month                  16
purchase_hour_section            4
purchase_day                     2
dtype: int64

In [18]:
transaction[category_cols] =transaction[category_cols].fillna(-1).astype(str)

# 3.进行特征提取

In [19]:
features = {}
columns = transaction.columns.tolist()
idx = columns.index('card_id')
category_cols_index = [columns.index(col) for col in category_cols]
numeric_cols_index = [columns.index(col) for col in numeric_cols]

for card in train['card_id'].values.tolist()+test['card_id'].values.tolist():
    features[card] = {}

month_idx = columns.index('month_lag')
import time
s = time.time()
num = 0
for i in range(transaction.shape[0]):
    va = transaction.loc[i].values
    card = va[idx]
    for cate_ind in category_cols_index:
        for num_ind in numeric_cols_index:
            col_name = '&'.join([columns[cate_ind], va[cate_ind], columns[num_ind]])
            features[card][col_name] = features[card].get(col_name, 0) + va[num_ind]
            if int(va[month_idx]) < 0:
                col_name = '&'.join([columns[cate_ind], va[cate_ind], columns[num_ind], 'hist'])
            else:
                col_name = '&'.join([columns[cate_ind], va[cate_ind], columns[num_ind], 'new'])
            features[card][col_name] = features[card].get(col_name, 0) + va[num_ind]
            
    num += 1
    if num%1000000==0:
        print(time.time()-s, "s")
del transaction
gc.collect()

307.89039421081543 s
567.0080506801605 s
824.4696021080017 s
1084.6360008716583 s
1344.127039194107 s
1601.6303725242615 s
1867.129280090332 s
2137.207996368408 s
2411.4278552532196 s
2679.254681825638 s
2950.939044713974 s
3222.253619670868 s
3486.751174211502 s
3756.52765083313 s
4032.572408914566 s
4301.938138484955 s
4569.598265171051 s
4840.817482948303 s
5111.80669760704 s
5384.62407040596 s
5662.289692640305 s
5941.815702199936 s
6224.922076702118 s
6509.909406661987 s
6794.85777759552 s
7081.087693929672 s
7372.041896104813 s
7664.8579359054565 s
7950.523193120956 s
8237.561739683151 s
8527.694004058838 s


80

In [20]:
df = pd.DataFrame(features).T
del features
df = df.reset_index()
cols = df.columns.tolist()
df.columns = ['card_id'] + cols[1:]
df.to_csv('preprocess/baseline_features_depart.csv', index=False)

# 4.进行模型训练及预测

In [21]:
train = pd.merge(train, df, how='left', on='card_id')
test =  pd.merge(test, df, how='left', on='card_id')
del df

label = 'target'
features = train.columns.tolist()
features.remove('card_id')
features.remove('target')

train = train.sample(frac=1, random_state=2020)
valid = train[int(0.8*train.shape[0]):].reset_index(drop=True)
train = train[:int(0.8*train.shape[0])].reset_index(drop=True)

In [22]:
# 配置模型的训练参数
params_initial = {
    'num_leaves':31, 
    'learning_rate':0.1, 
'boosting':'gbdt',
'min_child_samples':20,
'bagging_seed': 2020,
'bagging_fraction':0.7, 
'bagging_freq':1,
'feature_fraction':0.7, 
 'max_depth':-1,
    'metric':'rmse',
'reg_alpha':0,
'reg_lambda':1, 
'objective':'regression'
    }
ESR = 30
NBR = 10000
VBE = 50

# 转换训练集与验证集为指定的数据格式
train = lgb.Dataset(train[features], train[label])
valid = lgb.Dataset(valid[features], valid[label])

# 模型训练
bst = lgb.train(params_initial,train, 
      num_boost_round=NBR, valid_sets=[train,valid],
      valid_names=['train','valid'],early_stopping_rounds=ESR,verbose_eval=VBE)

# 预测结果
prediction = bst.predict(test[features])
# 获取模型的特征重要性
fse = pd.Series(bst.feature_importance(), index=features)


test['target'] = prediction

submission = test[['card_id', 'target']]


submission.to_csv('result/baseline_depart_submission.csv', index=False)

Training until validation scores don't improve for 30 rounds
[50]	train's rmse: 3.48243	valid's rmse: 3.61001
Early stopping, best iteration is:
[61]	train's rmse: 3.44729	valid's rmse: 3.60826


In [23]:
fse.to_csv('result/baseline_depart_feature_importance.csv', index=False)

In [24]:
# 后续优化方向
# 1. 只用最近的交易记录(效果较差)
# 2. 特征选择(LGB用特征重要性cv，RF用相关系数等进行筛选，XGB用Stacking进行降维)
# 3. 五折
# 4. 调参（LGB用超参调优，RF用GridSearch）
# 5. 模型融合(LGB+RF+XGB,类别特征少，因此不用CATBOOST)

In [25]:
fse.shape

(4126,)